# resampling

- 時系列データを `rule` に従ってリサンプルします。
- [pandas.DataFrame.resample — pandas 1.4.1 documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.resample.html)



In [35]:
import asyncio

import nest_asyncio
import pandas as pd
import plotly.graph_objects as go
import pybotters
from IPython.display import HTML

nest_asyncio.apply()

## API から Trade データを取得

- 取引データから OHLCV を作成します
- [Get trades – API Documentation](https://docs.ftx.com/#get-trades)


In [36]:
async def get_trades(market_name, start_time, end_time):
    async with pybotters.Client(
        apis={"ftx": ["", ""]}, base_url="https://ftx.com/api"
    ) as client:
        res = await client.get(
            f"/markets/{market_name}/trades",
            params={
                "start_time": start_time,
                "end_time": end_time,
            },
        )
        return await res.json()


data = asyncio.run(get_trades("BTC-PERP", 1643641200, 1643727600))
data["result"][:3]

[{'id': 3254047916,
  'price': 38526.0,
  'size': 0.1464,
  'side': 'buy',
  'liquidation': False,
  'time': '2022-02-01T14:59:59.690649+00:00'},
 {'id': 3254047915,
  'price': 38526.0,
  'size': 0.0536,
  'side': 'buy',
  'liquidation': False,
  'time': '2022-02-01T14:59:59.690649+00:00'},
 {'id': 3254047888,
  'price': 38524.0,
  'size': 0.0047,
  'side': 'sell',
  'liquidation': False,
  'time': '2022-02-01T14:59:59.288778+00:00'}]

## DataFrame の作成
- `time` コラムを datetimeindex に持つDataFrameを作成します

In [37]:
df = pd.DataFrame(data["result"])
df.head()

,id,price,size,side,liquidation,time
0,3254047916,38526.0,0.1464,buy,False,2022-02-01T14:59:59.690649+00:00
1,3254047915,38526.0,0.0536,buy,False,2022-02-01T14:59:59.690649+00:00
2,3254047888,38524.0,0.0047,sell,False,2022-02-01T14:59:59.288778+00:00
3,3254047870,38524.0,0.1210,sell,False,2022-02-01T14:59:59.078000+00:00
4,3254047855,38524.0,0.0127,sell,False,2022-02-01T14:59:58.777561+00:00


In [38]:
df.dtypes


id               int64
price          float64
size           float64
side            object
liquidation       bool
time            object
dtype: object

In [39]:
# time を datetime 型に変更し、この dataframe の index として設定
df = pd.DataFrame(data["result"])
df["time"] = pd.to_datetime(df["time"])
df.set_index("time", inplace=True)
df.sort_index(inplace=True)
df

,id,price,size,side,liquidation
time,,,,,
2022-02-01 14:53:19.339001+00:00,3253998374,38476.0,0.0014,buy,False
2022-02-01 14:53:19.370232+00:00,3253998377,38475.0,0.0080,sell,False
2022-02-01 14:53:19.370232+00:00,3253998375,38475.0,0.6960,sell,False
2022-02-01 14:53:19.370232+00:00,3253998376,38475.0,0.2960,sell,False
2022-02-01 14:53:19.423013+00:00,3253998379,38475.0,0.1500,sell,False
...,...,...,...,...,...
2022-02-01 14:59:58.777561+00:00,3254047855,38524.0,0.0127,sell,False
2022-02-01 14:59:59.078000+00:00,3254047870,38524.0,0.1210,sell,False
2022-02-01 14:59:59.288778+00:00,3254047888,38524.0,0.0047,sell,False


## Resample 
- [pandas.DataFrame.resample — pandas 1.4.1 documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.resample.html?highlight=resample#pandas.DataFrame.resample)
- `DatetimeIndex`, `PeriodIndex`, `TimedeltaIndex` を持つデータであれば、 `rule` に従ってリサンプルできます。
- `rule` はこちらを参照してください：[Time series / date functionality — pandas 1.4.1 documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects) 
- `label` は 区切った時間帯の「どちら側」をラベルにするかを指定します。（詳細は後述）


### 例：取引データから1分足と出来高を作成

In [40]:
# まずは dataframe の resample メソッドを使って、Resampler object を作成
df_resampled = df.resample("1min", label="right")
df_ohlc = df_resampled["price"].ohlc()
df_ohlc["volume"] = df_resampled["size"].sum()
df_ohlc

,open,high,low,close,volume
time,,,,,
2022-02-01 14:54:00+00:00,38476.0,38538.0,38452.0,38516.0,89.7768
2022-02-01 14:55:00+00:00,38517.0,38520.0,38383.0,38395.0,201.3913
2022-02-01 14:56:00+00:00,38395.0,38461.0,38374.0,38417.0,257.0133
2022-02-01 14:57:00+00:00,38416.0,38417.0,38336.0,38395.0,213.7127
2022-02-01 14:58:00+00:00,38395.0,38478.0,38394.0,38476.0,177.0822
2022-02-01 14:59:00+00:00,38472.0,38473.0,38407.0,38473.0,94.0939
2022-02-01 15:00:00+00:00,38473.0,38550.0,38473.0,38526.0,123.6707


### 詳しく説明

- TBA
